In [1]:

from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=1,
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)

examples = [
    {
        "question": "What was the movie I asked about earlier?",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "question": "What was the movie I asked about earlier?",
        "answer": "👨‍👨‍👦🔫🍝",
    },
    {
        "question": "What was the movie I asked about earlier?",
        "answer": "Pretty Woman",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [("human", "{question}"), ("ai", "{answer}")]
)

fewshot_chat_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """
     You are a helpful AI machine.
     
     If I ask, ‘What was the movie I asked about earlier?’, 
     respond with 🛩️👨‍✈️🔥 if it’s Top Gun, 👨‍👨‍👦🔫🍝 if it’s The Godfather, 
     and you don’t need to transform it for any other movies.
     """),
    fewshot_chat_prompt,
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({
        "question": question,
    })

    memory.save_context({"input": question}, {"output": result.content})

    print(result)

In [2]:
invoke_chain(question="Among the movies Tom Cruise appeared in, the one related to airplanes is Top Gun")

content='Got it!'


In [3]:
invoke_chain(question="What was the movie I asked about earlier?")

content='🛩️👨\u200d✈️🔥'


In [4]:
invoke_chain(question="Which movies starring Al Pacino are related to the mafia?")

content='The Godfather'


In [5]:
invoke_chain(question="What was the movie I asked about earlier?")

content='👨\u200d👨\u200d👦🔫🍝'
